## Probability a Single Facility in a Single Year Seeds a Pandemic:

$
P_{\text{single_pandemic}} = P_{\text{release}} \times P_{\text{seeds_pandemic}}
$

where:
- $P_{\text{release}}$ is the probability of community release from a single facility in a single year.
- $P_{\text{seeds_pandemic}}$ is the probability that a virus release seeds a pandemic. Since $P_{\text{seeds_pandemic}}$ is given as a range, we will sample from a uniform distribution between 0.05 and 0.4.

## Number of Pandemics Seeded by Any matHPAI Facility:

For each year, for each facility, we'll calculate the probability it seeds a pandemic using a Bernoulli trial with $P_{\text{single_pandemic}}$. Summing across all facilities and all years will give us the total number of pandemics seeded by any facility.

## Worldwide Fatalities:

For each pandemic, the number of fatalities is calculated as:

$
\text{fatalities} = \text{population} \times \text{infection_rate} \times \text{fatality_rate}
$

Summing across all pandemics will give the total fatalities.

In [ ]:
from ipywidgets import widgets, HBox, VBox, HTML, Layout, interactive
import numpy as np
import plotly.graph_objects as go

#################################
# Define parameters and functions
#################################

# Given parameters
P_release = 0.00246
P_seeds_pandemic_min = 0.05
P_seeds_pandemic_max = 0.4
num_years = 100
num_facilities = 14
population = 7.9e9
fatality_rate = 0.025
infection_rate = 0.15

# Monte Carlo simulations
num_simulations = 1000

def run_simulation(
        P_release=0.00246,
        P_seeds_pandemic_min=0.05,
        P_seeds_pandemic_max=0.4,
        num_years = 100,
        num_facilities = 14,
        population = 7.9e9,
        fatality_rate = 0.025,
        infection_rate = 0.15,
        num_simulations = 100000
    ):
    """
    Run a Monte Carlo simulation of the number of pandemics and fatalities
    """
    total_pandemics = []
    total_fatalities = []

    for _ in range(num_simulations):
        P_seeds_pandemic = np.random.uniform(P_seeds_pandemic_min, P_seeds_pandemic_max)
        P_single_pandemic = P_release * P_seeds_pandemic
        
        # Number of pandemics seeded by any facility in a single year
        pandemics_per_year = np.random.binomial(num_facilities, P_single_pandemic, num_years)
        total_num_pandemics = sum(pandemics_per_year)
        
        # Fatalities for each pandemic
        fatalities_per_pandemic = population * infection_rate * fatality_rate * total_num_pandemics
        
        total_pandemics.append(total_num_pandemics)
        total_fatalities.append(fatalities_per_pandemic)

    # Convert to numpy arrays for statistics
    total_pandemics = np.array(total_pandemics)
    total_fatalities = np.array(total_fatalities)

    # Calculate some statistics
    mean_pandemics = np.mean(total_pandemics)
    mean_fatalities = np.mean(total_fatalities)
    percentile_5_fatalities = np.percentile(total_fatalities, 5)
    percentile_95_fatalities = np.percentile(total_fatalities, 95)

    plot_histogram(total_fatalities, mean_fatalities, np.median(total_fatalities), percentile_5_fatalities, percentile_95_fatalities)

    return mean_pandemics, mean_fatalities, percentile_5_fatalities, percentile_95_fatalities

def plot_histogram(total_deaths, mean_deaths, median_deaths, percentile_5, percentile_95):
    """Plot an interactive histogram of expected deaths with mean, median, and percentiles."""

    # Create the interactive histogram using plotly
    fig = go.Figure()

    # Add histogram data
    fig.add_trace(go.Histogram(x=total_deaths, name='Deaths Distribution', 
                               xbins=dict(size=5e7), # bin size of 50 million
                               marker_color='skyblue'))

    # Add vertical lines for mean, median, and percentiles
    fig.add_shape(
        go.layout.Shape(type='line', x0=mean_deaths, x1=mean_deaths, y0=0, y1=1, yref='paper',
                        line=dict(color='red', dash='dash'))
    )
    fig.add_shape(
        go.layout.Shape(type='line', x0=median_deaths, x1=median_deaths, y0=0, y1=1, yref='paper',
                        line=dict(color='green', dash='dash'))
    )
    fig.add_shape(
        go.layout.Shape(type='line', x0=percentile_5, x1=percentile_5, y0=0, y1=1, yref='paper',
                        line=dict(color='blue', dash='dash'))
    )
    fig.add_shape(
        go.layout.Shape(type='line', x0=percentile_95, x1=percentile_95, y0=0, y1=1, yref='paper',
                        line=dict(color='yellow', dash='dash'))
    )

    # Add legend and labels
    fig.update_layout(
        title="Distribution of Expected Deaths from Accidental Pandemics This Century",
        xaxis_title="Number of Deaths",
        yaxis_title="Count",
        shapes=[
            dict(type='line', xref='x', yref='paper',
                 x0=mean_deaths, y0=0, x1=mean_deaths, y1=1, line=dict(color='red', dash='dash'),
                 name='Mean'),
            dict(type='line', xref='x', yref='paper',
                 x0=median_deaths, y0=0, x1=median_deaths, y1=1, line=dict(color='green', dash='dash'),
                 name='Median'),
            dict(type='line', xref='x', yref='paper',
                 x0=percentile_5, y0=0, x1=percentile_5, y1=1, line=dict(color='blue', dash='dash'),
                 name='5th Percentile'),
            dict(type='line', xref='x', yref='paper',
                 x0=percentile_95, y0=0, x1=percentile_95, y1=1, line=dict(color='yellow', dash='dash'),
                 name='95th Percentile')
        ],
        legend_title_text='Statistics'
    )

    fig.show()

## Parameters

In [ ]:
# Source link
klotz_2021 = "https://armscontrolcenter.org/wp-content/uploads/2017/04/LWC-paper-final-version-for-CACNP-website.pdf"

# Define a consistent layout for the description widgets
desc_layout = widgets.Layout(width='600px')

# Create HTML widgets for descriptions with links
# Global variables
num_years_desc = HTML(value="Number of years", layout=desc_layout)
population_desc = HTML(value="World population", layout=desc_layout)
# Accidental release
P_release_desc = HTML(
    value=f"Probability of community release from a single facility (with one or many labs) in a single year (default estimated in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
P_seeds_pandemic_min_desc = HTML(
    value=f"Probability Virus Seeds Pandemic (Min) (default value estimate in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
P_seeds_pandemic_max_desc = HTML(
    value=f"Probability Virus Seeds Pandemic (Max) (default value estimate in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
num_facilities_desc = HTML(
    value=f"Number of Facilities (default is number of HPAI facilities as stated in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
fatality_rate_desc = HTML(
    value=f"Case fatality rate (default is 1918 influenza CFR, <a href='{klotz_2021}'>see Klotz 2021</a>):", 
    layout=desc_layout
)
infection_rate_desc = HTML(
    value=f"Fraction of Population Infected (default value: % infected in typical flu season, <a href='{klotz_2021}'>see Klotz 2021</a>):", 
    layout=desc_layout
)

# Create text widgets without descriptions 
num_years_widget = widgets.IntText(value=100, layout=widgets.Layout(width='200px'))
population_widget = widgets.FloatText(value=7.9e9, layout=widgets.Layout(width='200px'))
P_release_widget = widgets.FloatText(value=0.00246, layout=widgets.Layout(width='200px'))
P_seeds_pandemic_min_widget = widgets.FloatText(value=0.05, layout=widgets.Layout(width='200px'))
P_seeds_pandemic_max_widget = widgets.FloatText(value=0.4, layout=widgets.Layout(width='200px'))
num_facilities_widget = widgets.IntText(value=14, layout=widgets.Layout(width='200px'))
fatality_rate_widget = widgets.FloatText(value=0.025, layout=widgets.Layout(width='200px'))
infection_rate_widget = widgets.FloatText(value=0.15, layout=widgets.Layout(width='200px'))

# Define the layout for the subtitles
subtitle_layout = Layout(padding='0px 0px 0px 0px', font_size='1.2em')
global_variables_subtitle = HTML(value="<strong>Global Variables</strong>")
accidental_release_subtitle = HTML(value="<strong>Accidental Release</strong>")

# Group the widgets
global_variables_widgets = VBox([
    global_variables_subtitle,
    HBox([num_years_desc, num_years_widget]),
    HBox([population_desc, population_widget]),
])

accidental_release_widgets = VBox([
    accidental_release_subtitle,
    HBox([P_release_desc, P_release_widget]),
    HBox([P_seeds_pandemic_min_desc, P_seeds_pandemic_min_widget]),
    HBox([P_seeds_pandemic_max_desc, P_seeds_pandemic_max_widget]),
    HBox([num_facilities_desc, num_facilities_widget]),
    HBox([fatality_rate_desc, fatality_rate_widget]),
    HBox([infection_rate_desc, infection_rate_widget]),
])

In [ ]:
# Create an interactive widget
interactive_plot = interactive(run_simulation, 
                              P_release=P_release_widget,
                              P_seeds_pandemic_min=P_seeds_pandemic_min_widget,
                              P_seeds_pandemic_max=P_seeds_pandemic_max_widget,
                              num_years=num_years_widget,
                              num_facilities=num_facilities_widget,
                              population=population_widget,
                              fatality_rate=fatality_rate_widget,
                              infection_rate=infection_rate_widget,
                              num_simulations=num_simulations)

# Display widgets and the output
output = interactive_plot.children[-1]
display(global_variables_widgets)
display(accidental_release_widgets)
run_simulation()
display(output)